In [52]:
import panel as pn
pn.extension()
import pandas as pd
import numpy as np
import hvplot.pandas
import param, random

# Helpers
HOURS = 60 * 60
DAYS = 24 * HOURS
WEEKS = 7 * DAYS
ONE_HUNDRED_PERCENT = 1e18
ONE_TOKEN = 1e18
FUNDRAISING_ONE_HUNDRED_PERCENT = 1e6
FUNDRAISING_ONE_TOKEN = 1e18
HOLDERS_PER_TRANSACTION = 10
PPM = 1000000
BLOCKTIME = 5

In [27]:
# This is the amount of wxDAI that can be staked to create TEC hatch tokens - TEC hatch upgrades into a commons
HATCH_ORACLE_RATIO = 0.005 

In [19]:
# Dandelion Parameters
SUPPORT_REQUIRED = 0.6
MIN_ACCEPTANCE_QUORUM = 0.02
VOTE_DURATION_BLOCKS = 3 * DAYS / BLOCKTIME
VOTE_BUFFER_BLOCKS = 8 * HOURS / BLOCKTIME
VOTE_EXECUTION_DELAY_BLOCKS = 24 * HOURS / BLOCKTIME
VOTING_SETTINGS = [SUPPORT_REQUIRED, MIN_ACCEPTANCE_QUORUM, VOTE_DURATION_BLOCKS, VOTE_BUFFER_BLOCKS, VOTE_EXECUTION_DELAY_BLOCKS]

# Set the fee paid to the org to create an administrative vote
TOLLGATE_FEE = 3 * ONE_TOKEN

In [20]:
# How many tokens required to initialize the bonding curve
HATCH_MIN_GOAL = 100
HATCH_MAX_GOAL = 100*1e6

In [21]:
# How long should the presale period last for
PRESALE_PERIOD = 7 * DAYS

# How many organization tokens per collateral token should be minted
PRESALE_EXCHANGE_RATE = 0.00000001 * FUNDRAISING_ONE_TOKEN

# When is the cliff for vesting restrictions
VESTING_CLIFF_PERIOD = PRESALE_PERIOD + 3 * DAYS # 3 days after presale

# When will pre-sale contributors be fully vested
VESTING_COMPLETE_PERIOD = VESTING_CLIFF_PERIOD + 3 * WEEKS # 3 weeks after cliff
PRESALE_PERCENT_SUPPLY_OFFERED = FUNDRAISING_ONE_HUNDRED_PERCENT

# What percentage of pre-sale contributions should go to the common pool (versus the reserve)
PRESALE_PERCENT_FUNDING_FOR_BENEFICIARY = 0.35 * FUNDRAISING_ONE_HUNDRED_PERCENT

# when should the pre-sale be open, setting 0 will allow anyone to open the pre-sale anytime after deployment
OPEN_DATE = 0

In [83]:
import param
import pandas as pd
import datetime as dt

class Hatch(param.Parameterized):
    # CSTK Ratio
    total_cstk_tokens = param.Number(700000, constant=True)
    hatch_oracle_ratio = param.Number(0.005, constant=True)
    @param.depends('hatch_oracle_ratio', 'total_cstk_tokens')
    def wxdai_range(self):
        return pn.Row(pn.Pane("Cap on wxdai staked: "), self.hatch_oracle_ratio * self.total_cstk_tokens)
    
    # Dandelion Params
    support_required = param.Integer(50000, bounds=(-200, 100000))
    
    # Min and Target Goals
    min_goal = param.Number(1000, bounds=(1000,1e6), step=1000)
    target_goal = param.Number(100000, bounds=(100000,1e8), step=10000)
    #pn.widgets.FloatSlider(name='Max Goal', start=min_goal.value, end=1e8, step=10000, value=min_goal.value)


In [84]:
import panel as pn

In [85]:
hatch = Hatch()

In [86]:
pn.Row(hatch, hatch.wxdai_range)

Row
    [0] Column(margin=5, name='Hatch', width=300)
        [0] StaticText(value='<b>Hatch</b>')
        [1] FloatInput(disabled=True, name='Total cstk tokens', value=700000, value_throttled=700000)
        [2] FloatInput(disabled=True, name='Hatch oracle ratio', value=0.005, value_throttled=0.005)
        [3] IntSlider(end=100000, name='Support required', start=-200, value=50000, value_throttled=50000)
        [4] FloatSlider(end=1000000.0, name='Min goal', start=1000, step=1000, value=1000, value_throttled=1000)
        [5] FloatSlider(end=100000000.0, name='Target goal', start=100000, step=10000, value=100000, value_throttled=100000)
    [1] ParamMethod(method)

### Min Goal Max Goal

In [44]:



@pn.depends(min_goal, max_goal)
def dashboard(min_goal, max_goal):
    df = pd.DataFrame(np.linspace(min_goal,max_goal))
    return df.hvplot.line()

pn.Column(min_goal, max_goal, dashboard)

Column
    [0] FloatSlider(end=50000, name='Min Goal', start=1000, step=10000, value=1000, value_throttled=1000)
    [1] FloatSlider(end=100000000.0, name='Max Goal', start=1000, step=10000, value=1000, value_throttled=1000)
    [2] ParamFunction(function)

In [46]:
cstk_ratio = pn.widgets.FloatSlider(name='CSTK Ratio', start=0.001, end=0.01, step=0.001, value=0.001)
total_cstk_tokens = 700000 

@pn.depends(cstk_ratio)
def wxdai_range(cstk_ratio):
    return pn.Row(pn.Pane("Cap on wxdai staked: "), cstk_ratio * total_cstk_tokens)

In [47]:
pn.Column(cstk_ratio, wxdai_range)

Column
    [0] FloatSlider(end=0.01, name='CSTK Ratio', start=0.001, step=0.001, value=0.001, value_throttled=0.001)
    [1] ParamFunction(function)